The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [44]:
!pip install d2l==1.0.0-alpha1.post0


# Synthetic Regression Data
:label:`sec_synthetic-regression-data`


Machine learning is all about extracting information from data.
So you might wonder, what could we possibly learn from synthetic data?
While we might not care intrinsically about the patterns 
that we ourselves baked into an artificial data generating model,
such datasets are nevertheless useful for didactic purposes,
helping us to evaluate the properties of our learning 
algorithms and to confirm that our implementations work as expected.
For example, if we create data for which the correct parameters are known *a priori*,
then we can verify that our model can in fact recover them.


In [45]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

## Generating the Dataset

For this example, we will work low-dimensional
for succinctness.
The following code snippet generates 1000 examples
with 2-dimensional features drawn 
from a standard normal distribution.
The resulting design matrix $\mathbf{X}$
belongs to $\mathbb{R}^{1000 \times 2}$. 
We generate each label by applying 
a *ground truth* linear function, 
corrupted them via additive noise $\epsilon$, 
drawn independently and identically for each example:

$$
\mathbf{y}= \mathbf{X} \mathbf{w} + b + \mathbf\epsilon.
$$

For convenience we assume that $\epsilon$ is drawn 
from a normal distribution with mean $\mu= 0$ 
and standard deviation $\sigma = 0.01$.

- Note that for object-oriented design
we add the code to the `__init__` method of a subclass of `d2l.DataModule` (introduced in :numref:`oo-design-data`). 

- It's good practice to allow setting any additional hyperparameters. 
We accomplish this with `save_hyperparameters()`. 
The `batch_size` will be determined later on.


Below, we set the true parameters to $\mathbf{w} = [2, -3.4]^\top$ and $b = 4.2$.
Later, we can check our estimated parameters against these *ground truth* values.


In [47]:
data = SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)

In [48]:
data.X, data.y

(tensor([[ 0.1910, -1.0912],
         [ 1.0457, -0.4085],
         [-1.9408,  0.6971],
         ...,
         [ 2.0278,  1.3265],
         [-0.7992, -0.1825],
         [-1.3803, -0.8651]]),
 tensor([[ 8.2957],
         [ 7.6946],
         [-2.0524],
         ...,
         [ 3.7484],
         [ 3.2150],
         [ 4.3921]]))

In [96]:
class SyntheticRegressionData(d2l.DataModule):  #@save
    def __init__(self, w, b, noise=0.01, num_train=1000, num_val=1000,
                 batch_size=32):
        super().__init__()
        self.save_hyperparameters()
        n = num_train + num_val
        self.X = torch.randn(n, len(w))
        noise = torch.randn(n, 1) * noise
        self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise
data = SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)
data.noise

0.01

Each row in `features` consists of a vector in $\mathbb{R}^2$ and each row in `labels` is a scalar.
Let's have a look at the first entry.

In [97]:
print('features:', data.X[0],'\nlabel:', data.y[0])

features: tensor([-0.4104,  0.1957]) 
label: tensor([2.7090])


## Reading the Dataset

- Training machine learning models often requires multiple passes over a dataset,
grabbing one minibatch of examples at a time.
This data is then used to update the model.

- To illustrate how this works, 
we implement the `get_dataloader` function,
registering it as a method in the `SyntheticRegressionData` class via `add_to_class` 
(introduced in :numref:`oo-design-utilities`).

- It takes a batch size, a matrix of features,
and a vector of labels, and generates minibatches of size `batch_size`.
As such, each minibatch consists of a tuple of features and labels. 

- Note that we need to be mindful of whether we're in training or validation mode: 
in the former, 
we will want to read the data in random order, 
whereas for the latter,
being able to read data in a pre-defined order may be important for debugging purposes.


In [98]:
@d2l.add_to_class(SyntheticRegressionData)
def get_dataloader(self, train):
    if train:
        indices = list(range(0, self.num_train))
        # The examples are read in random order
        random.shuffle(indices)
    else:
        indices = list(range(self.num_train, self.num_train+self.num_val))
    for i in range(0, len(indices), self.batch_size):
        batch_indices = torch.tensor(indices[i: i+self.batch_size])
        yield self.X[batch_indices], self.y[batch_indices]

- https://ithelp.ithome.com.tw/articles/10258195

In [99]:
def yield_test(n):
    print("start n =", n)
    for i in range(n):
        yield i*i
        print("i =", i) # 最後一項時只會執行到 yield 不會繼續下去，所以後面的
    print("end")
tests = yield_test(3)

- 執行三次可看出作用

In [100]:
next(tests)

start n = 3


0

In [101]:
next(tests)

i = 0


1

In [102]:
next(tests)

i = 1


4

In [103]:
next(tests)

i = 2
end


StopIteration: 

In [107]:
for test in tests:
    print("yield: ", test)
    print("--------")
tests = yield_test(3)
for test in tests:
    print("yield: ", test)
    print("--------")

start n = 3
yield:  0
--------
i = 0
yield:  1
--------
i = 1
yield:  4
--------
i = 2
end


To build some intuition, 
let's inspect the first minibatch of data. 
Each minibatch of features provides us with both its size and the dimensionality of input features.
Likewise, our minibatch of labels will have a matching shape given by `batch_size`.


In [106]:
next(data.train_dataloader())

(tensor([[-1.7844,  0.4149],
         [-0.1671, -0.2875],
         [-1.4256,  0.9691],
         [-0.3123,  0.0596],
         [-0.1961,  0.0438],
         [-0.1566, -1.3969],
         [ 0.3577, -2.1297],
         [ 0.2844, -0.3421],
         [ 0.1227,  1.0142],
         [-0.1211,  0.6842],
         [ 1.1255, -0.3765],
         [-1.7070,  1.1289],
         [-0.7528,  0.4416],
         [ 0.1696,  0.1785],
         [ 0.5595,  0.1191],
         [-1.3201, -0.3713],
         [-0.8688,  0.4932],
         [ 0.7638,  0.0989],
         [ 0.5768,  0.4609],
         [-0.5516,  0.5005],
         [-0.1589,  0.1175],
         [ 0.1363,  0.0227],
         [-0.6584,  0.0261],
         [-2.1747,  1.4089],
         [ 0.5673,  0.0193],
         [ 0.9940, -0.2829],
         [-1.4849, -0.5273],
         [-0.8087,  1.0222],
         [ 1.5681,  0.8228],
         [ 0.0061, -0.1476],
         [-0.6502, -0.0146],
         [ 0.7585, -1.0735]]),
 tensor([[-0.7862],
         [ 4.8398],
         [-1.9520],
         [

In [108]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


In [109]:
next(data.train_dataloader())

(tensor([[ 0.6166,  0.7362],
         [-2.1747,  1.4089],
         [ 0.3780,  1.4513],
         [-0.6670, -0.3031],
         [-0.4569, -0.3421],
         [ 1.4955, -0.2919],
         [ 0.7895, -0.8221],
         [-0.2102,  1.9559],
         [-1.2427, -0.4630],
         [ 0.7638,  0.0989],
         [-0.0209, -0.1050],
         [-0.8688,  0.4932],
         [ 1.4120, -1.0275],
         [-0.8697, -0.5213],
         [-0.9751,  1.1446],
         [-1.5455,  0.9903],
         [ 2.3733,  0.0521],
         [-0.5018, -0.1240],
         [-2.2299,  1.8953],
         [-0.2458,  1.6103],
         [-0.0595, -0.0132],
         [-0.9496, -0.6938],
         [ 1.5717,  0.2121],
         [-0.4800, -0.5913],
         [ 2.4407,  0.8867],
         [ 0.6126, -0.7681],
         [-0.8624,  0.7481],
         [-0.7180,  0.8703],
         [-0.4481, -1.3920],
         [-1.9863, -0.6960],
         [ 0.7544,  0.0977],
         [ 0.8523,  1.4298]]),
 tensor([[ 2.9300],
         [-4.9400],
         [ 0.0208],
         [

[迭代器與生成器](https://www.runoob.com/python3/python3-iterator-generator.html)

In [113]:
List=[1,2,3,4]
it = iter(List)    # 创建迭代器对象
next(it)
next(it)

2

In [114]:
next(it)

3

In [115]:
next(it)

4

In [116]:
next(it)

StopIteration: 

While seemingly innocuous, the invocation of `iter(data.train_dataloader())` 
illustrates the power of Python's object-oriented design. 
Note that we added a method to the `SyntheticRegressionData` class
*after* creating the `data` object. 
Nonetheless, the object benefits from 
the *ex post facto* addition of functionality to the class.

Throughout the iteration we obtain distinct minibatches until the entire dataset has been exhausted (try this).
While the iteration implemented above is good for didactic purposes,
it is inefficient in ways that might get us in trouble on real problems.
For example, it requires that we load all the data in memory
and that we perform lots of random memory access.
The built-in iterators implemented in a deep learning framework
are considerably more efficient and they can deal
with sources such as data stored in files, 
data received via a stream, 
and data generated or processed on the fly. 
Next let's try to implement the same function using built-in iterators.

## Concise Implementation of the Data Loader

Rather than writing our own iterator,
we can call the existing API in a framework to load data.

As before, we need a dataset with features `X` and labels `y`. 

Beyond that, we set `batch_size` in the built-in data loader 
and let it take care of shuffling examples  efficiently.


- https://www.runoob.com/python/python-func-slice.html

In [125]:
myslice = slice(None,None,3)
for i in range(10)[myslice]:
    print(i)

0
3
6
9


- https://pytorch.org/docs/stable/data.html

In [117]:
@d2l.add_to_class(d2l.DataModule)  #@save
def get_tensorloader(self, tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors)
    dataset = torch.utils.data.TensorDataset(*tensors)
    return torch.utils.data.DataLoader(dataset, self.batch_size,
                                       shuffle=train)
@d2l.add_to_class(SyntheticRegressionData)  #@save
def get_dataloader(self, train):
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader((self.X, self.y), train, i)

The new data loader behaves just as the previous one, except that it is more efficient and has some added functionality.


In [ ]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


For instance, the data loader provided by the framework API 
supports the built-in `__len__` method, 
so we can query its length, 
i.e., the number of batches.


In [ ]:
len(data.train_dataloader())

32

## Summary

Data loaders are a convenient way of abstracting out 
the process of loading and manipulating data. 

This way the same machine learning *algorithm* 
is capable of processing many different types and sources of data 
without the need for modification. 

One of the nice things about data loaders 
is that they can be composed. 

For instance, we might be loading images 
and then have a post-processing filter 
that crops them or modifies them otherwise. 

As such, data loaders can be used 
to describe an entire data processing pipeline. 

As for the model itself, the two-dimensional linear model 
is about as simple a model as we might encounter. 

It lets us test out the accuracy of regression models 
without worry about having insufficient amounts of data 
or an underdetermined system of equations. 

We will put this to good use in the next section.  


## Exercises

1. What will happen if the number of examples cannot be divided by the batch size. How to change this behavior by specifying a different argument by using framework's API?
1. What if we want to generate a huge dataset, where both the size of the parameter vector `w` and the number of examples `num_examples` are large? 
    1. What happens if we cannot hold all data in memory?
    1. How would you shuffle the data if data is held on disk? Your task is to design an *efficient* algorithm that does not require too many random reads or writes. Hint: [pseudorandom permutation generators](https://en.wikipedia.org/wiki/Pseudorandom_permutation) allow you to design a reshuffle without the need to store the permutation table explicitly :cite:`Naor.Reingold.1999`. 
1. Implement a data generator that produces new data on the fly, every time the iterator is called. 
1. How would you design a random data generator that generates *the same* data each time it's called?


[Discussions](https://discuss.d2l.ai/t/6663)
